<a href="https://colab.research.google.com/github/JHyunjun/torch_1D-CNN/blob/main/1D_CNN_AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import tensorflow as tf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
tf.random.set_seed(1)

#Data Load
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt') 
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/test_v.txt')
df_train2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt') 
train_data_input = df_train.iloc[:,:]
train_data_output = df_train.iloc[:,:]
test_data_input = df_test.iloc[:,:]
test_data_output = df_test.iloc[:,:]
print(train_data_input.shape,train_data_input)
print(train_data_output.shape, train_data_output)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
time1 = 5500
time2 = 5800
#train_gan = df_train.iloc[0:len(df_train),-1]
train_gan = df_train.iloc[time1:time2, :].values
print(train_gan.shape)
train_gan2 = df_test.iloc[time1:time2, :].values
#train_gan2 = train_gan2.to_numpy()
trainSet_for_result = df_train2.iloc[time1:time2,:].values
print(train_gan)

In [ ]:
#Normalization

#print("Before Norm : ",train_data_input.shape, train_data_input)

def origin_minmax(data) : 
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])
  print(arr_min)
  print(data.shape[1])
  for i in range(data.shape[1]) : 
    print(i)
    max = np.max(data[:,i])
    min = np.min(data[:,i]) 
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max1, arr_min1 = origin_minmax(train_gan)
arr_max2, arr_min2 = origin_minmax(train_gan2)
#print(arr_max1, arr_min1)

def MinMaxScaler(data):
  for i in range(data.shape[1]) : 
    max = np.max(data[:,i])
    min = np.min(data[:,i])
    data[:,i] = (data[:,i] - min) / (max - min + 1e-7)
    pass
  return data

train_gan = MinMaxScaler(train_gan)
#train_gan2 = MinMaxScaler(train_gan2)

def back_MinMax(data,max,min):
  print(max)
  print(min)
  for i in range(data.shape[1]) : 
    data[:,i] = data[:,i] * (max[i] - min[i]) + min[i]
    pass
  return data

#print("After Norm : ",train_gan.shape, train_gan)
#print("After De_Norm : ",back_MinMax(train_gan, arr_max1, arr_min1))
train_gan = torch.tensor([train_gan], dtype = torch.float32)
train_gan2 = torch.tensor([train_gan2], dtype = torch.float32)
print(train_gan)
print("TrainSet for Result : ",trainSet_for_result)

In [ ]:
# Windowing for Sequence
'''
seq_length = 2

def sliding_window(time_series, seq_length):
    time_series = np.squeeze(time_series)
    dataX = []
    dataY = []
    for i in range(0, int(len(time_series)/seq_length)): 
        _x = time_series[(i*seq_length):(i*seq_length) + seq_length, :]
        _y = time_series[(i*seq_length):(i*seq_length) + seq_length, :]  # for AE
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

train_gan_window_X, train_gan_window_Y = sliding_window(train_gan, seq_length)
test_gan_window_X, train_gan_window_Y = sliding_window(train_gan2, seq_length)

train_gan_window_X = torch.tensor(train_gan_window_X)
#print(train_gan_window_X.shape)
#print(train_gan_window_Y.shape)

train_gan_window_X = torch.tensor(train_gan_window_X, dtype = torch.float32)
train_gan_window_Y = torch.tensor(train_gan_window_Y, dtype = torch.float32)

print(train_gan.shape, train_gan)
print(train_gan_window_X.shape, train_gan_window_X)
'''

In [ ]:
#train_gan = train_gan.transpose(1,2)
#train_gan2 = train_gan2.transpose(1,2)
'''
print(train_gan_window_X.shape) 
print("train_gan_window_X.shape[0] : ",train_gan_window_X.shape[0])
print("train_gan_window_X.shape[1] : ",train_gan_window_X.shape[1])
print("train_gan_window_X.shape[2] : ",train_gan_window_X.shape[2])
'''

In [ ]:
#1D-CNN AE Class
filters = 10
kernel_size = 5
padding_size = 2
stride = 1

class CNN(nn.Module): 

  def __init__(self) : 

    super().__init__()

    self.a = nn.Conv1d(in_channels = seq_length, out_channels = filters, kernel_size = kernel_size, padding = padding_size, padding_mode = 'zeros')
    self.b = nn.LeakyReLU(0.02)
    self.c = nn.Linear(filters*5, 5*seq_length)
    self.d = nn.Sigmoid()

    self.loss_function = nn.MSELoss()
    self.optimiser = torch.optim.Adam(self.parameters(), lr = 1e-3)
    self.progress = []
    pass

  def forward(self, inputs) : 
     #simply run model
      print("Inputs : ", inputs.shape)
      x = self.a(inputs)
      #print("A", x.shape)
      x = self.b(x)
      #print("B", x.shape)
      x = torch.flatten(x)
      #print("Flatten : ", x.shape)
      x = self.c(x)
      #print("C", x.shape)
      #x = x.reshape(5, seq_length)
      x = x.reshape(seq_length, 5)
      print("After reshape : ",x.shape)
      x = self.d(x)
      print("D_Final Output", x)
      return x

  def train(self, inputs) :
    self.optimiser.zero_grad()
    #print("1")
    outputs = self.forward(inputs)
    #print("2")
    #print("input : ",inputs)
    #print("output : ",outputs.shape, outputs)
    loss = self.loss_function(inputs, outputs)
    #print("3")
    loss.backward()
    #print('4')
    self.optimiser.step()
    #print("5")

  def plot_progress(self) : 
    df = pd.DataFrame(self.progress, columns = ['1D CNN AE Loss'])
    df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))
    pass


In [ ]:
epochs1 = 10
epochs2 = 10
D = CNN()
seq_length = 3

for j in range(epochs1) : 
  for i in range(int((time2-time1)/seq_length)) :
    #refs = train_gan[0,:,i:i+seq_length]
    refs = train_gan[0,i:i+seq_length,:]
    #print("refs.shape : ",refs.shape) #[Features(5), Data_seq(3)]
    #print(" Training Input : ",refs)
    for j in range(epochs2) : 
      D.train(refs)
  pass
  pass
print("Percentage : ", j/epochs1 * 100, '%')
pass

In [ ]:
for i in range(int((time2-time1)/seq_length)-1) : 
  seed_test = train_gan[:, i:i+seq_length, :]
  Real_Value = trainSet_for_result[i:i+seq_length,:]
  #print(seed_test.shape)
  out_test = D.forward(seed_test)
  out_test = out_test.detach().cpu().numpy()
  out_test = out_test.reshape([seq_length,5])
  out_test = back_MinMax(out_test, arr_max1, arr_min1)
  print("Generated Value : ", out_test)
  print("Real Value : ", Real_Value)
  print("                                   ")

In [ ]:
print(trainSet_for_result.shape)